In [1]:
import pandas as pd
   
df = pd.read_csv('recipes.csv')

In [2]:
print(df.head())
print(df.info())
   

   RecipeId                               Name  AuthorId      AuthorName  \
0        38  Low-Fat Berry Blue Frozen Dessert      1533          Dancer   
1        39                            Biryani      1567        elly9812   
2        40                      Best Lemonade      1566  Stephen Little   
3        41     Carina's Tofu-Vegetable Kebabs      1586         Cyclopz   
4        42                       Cabbage Soup      1538       Duckie067   

  CookTime PrepTime TotalTime         DatePublished  \
0    PT24H    PT45M  PT24H45M  1999-08-09T21:46:00Z   
1    PT25M     PT4H   PT4H25M  1999-08-29T13:12:00Z   
2     PT5M    PT30M     PT35M  1999-09-05T19:52:00Z   
3    PT20M    PT24H  PT24H20M  1999-09-03T14:54:00Z   
4    PT30M    PT20M     PT50M  1999-09-19T06:19:00Z   

                                         Description  \
0  Make and share this Low-Fat Berry Blue Frozen ...   
1  Make and share this Biryani recipe from Food.com.   
2  This is from one of my  first Good House 

In [3]:
df = df[['Name', 'RecipeIngredientParts', 'RecipeInstructions']]

df.dropna(inplace=True)
   
print(df)

                                                    Name  \
0                      Low-Fat Berry Blue Frozen Dessert   
1                                                Biryani   
2                                          Best Lemonade   
3                         Carina's Tofu-Vegetable Kebabs   
4                                           Cabbage Soup   
...                                                  ...   
522512                    Meg's Fresh Ginger Gingerbread   
522513  Roast Prime Rib au Poivre with Mixed Peppercorns   
522514                             Kirshwasser Ice Cream   
522515          Quick & Easy Asian Cucumber Salmon Rolls   
522516                           Spicy Baked Scotch Eggs   

                                    RecipeIngredientParts  \
0       c("blueberries", "granulated sugar", "vanilla ...   
1       c("saffron", "milk", "hot green chili peppers"...   
2       c("sugar", "lemons, rind of", "lemon, zest of"...   
3       c("extra firm tofu", "eggpl

In [4]:
df['combined'] = df['Name'] + "\nIngredients:\n" + df['RecipeIngredientParts'] + "\nInstructions:\n" + df['RecipeInstructions']

print(df)

                                                    Name  \
0                      Low-Fat Berry Blue Frozen Dessert   
1                                                Biryani   
2                                          Best Lemonade   
3                         Carina's Tofu-Vegetable Kebabs   
4                                           Cabbage Soup   
...                                                  ...   
522512                    Meg's Fresh Ginger Gingerbread   
522513  Roast Prime Rib au Poivre with Mixed Peppercorns   
522514                             Kirshwasser Ice Cream   
522515          Quick & Easy Asian Cucumber Salmon Rolls   
522516                           Spicy Baked Scotch Eggs   

                                    RecipeIngredientParts  \
0       c("blueberries", "granulated sugar", "vanilla ...   
1       c("saffron", "milk", "hot green chili peppers"...   
2       c("sugar", "lemons, rind of", "lemon, zest of"...   
3       c("extra firm tofu", "eggpl

In [5]:
from sklearn.model_selection import train_test_split
   
train_data, test_data = train_test_split(df['combined'], test_size=0.2, random_state=42)

In [6]:
#step 2 - model selection - gpt 2
!pip install transformers

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch


In [54]:
#model

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


def generate_recipe(dish_name, dietary_restriction=None, max_length=200):
    
    prompt = f"Generate a detailed recipe for {dish_name} that is {dietary_restriction}. Include a list of ingredients and step-by-step instructions."

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,  
            early_stopping=True,
            temperature=0.7,  
            top_k=50,  
            top_p=0.95  
        )

    recipe = tokenizer.decode(output[0], skip_special_tokens=True)
    
    recipe = recipe.replace(prompt, "").strip()
    return recipe

dish_name = input("Enter the name of the dish you want a recipe for: ")
dietary_restriction = input("Do you have any dietary restrictions (e.g., vegetarian, gluten-free)? ")


generated_recipe = generate_recipe(dish_name, dietary_restriction)
print("\nGenerated Recipe:\n")
print(generated_recipe)


Enter the name of the dish you want a recipe for:  pasta
Do you have any dietary restrictions (e.g., vegetarian, gluten-free)?  veg


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Recipe:

Step 1: Prepare the pasta. In a large bowl, combine the olive oil, salt, pepper, and garlic. Add the spaghetti and cook until the sauce is thickened and the noodles are tender. Remove from the heat and set aside. Heat the oil in a small saucepan over medium-high heat. Bring the mixture to a boil, then reduce the temperature to low. Reduce the flame to medium and simmer for 10 minutes, or until sauce has thickens. Drain the liquid off the pan and let cool completely. Once the water has cooled, add the remaining pasta and stir to combine. Pour the rest of the ingredients into a sauce pan, cover with plastic wrap, let sit for 5 minutes and then remove from heat, remove the lid, place on a wire rack, set on the counter, for 30 minutes. Serve immediately.
